In [ ]:
# ======================= IMPORTS =======================
from google import genai
from google.genai import types
from google.colab import drive

import os, json, time, random, wave, base64
from datetime import datetime
from pathlib import Path

import torch
import librosa
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

# ======================= MOUNT DRIVE =======================
drive.mount('/content/drive')

# ======================= API KEY ROTATION =======================
KEYS_FILE = "/content/drive/MyDrive/Alot1.txt"

with open(KEYS_FILE) as f:
    API_KEYS = [k.strip() for k in f if k.strip()]

assert API_KEYS, "❌ No API keys found!"

current_key_index = 0
client = None

def configure_key():
    global client
    client = genai.Client(api_key=API_KEYS[current_key_index])
    print(f"🔑 Using API key {current_key_index + 1}/{len(API_KEYS)}")

def rotate_key():
    global current_key_index
    current_key_index += 1
    if current_key_index >= len(API_KEYS):
        raise RuntimeError("🛑 All API keys exhausted!")
    configure_key()

configure_key()

# ======================= PATHS =======================
DATASET_DIR = "/content/drive/MyDrive/teacher_dataset_large_updated"
VOICE_DIR = os.path.join(DATASET_DIR, "voices")
METADATA_FILE = os.path.join(DATASET_DIR, "metadata.jsonl")
os.makedirs(VOICE_DIR, exist_ok=True)

# ======================= VOICES & STYLES =======================
VOICE_MAP = {
    "Puck": {},
    "Leda": {},
    "Zephyr": {},
    "Kore": {},
    "Charon": {},
    "Aoede": {},
    "Gacrux": {},
    "Achird": {},
    "Sulafat": {},
    "Orus": {}
}

VOICES = list(VOICE_MAP.keys())

STYLES = [
    "cheerful","gentle","energetic","whispering","authoritative",
    "playful","calm","excited","sad","serious","friendly",
    "enthusiastic","soothing","firm","formal","curious",
    "happy","angry","fearful","neutral"
]

# ======================= CLEAN TEMPLATES (GENDER + STYLE ONLY) =======================
TEMPLATES = [
    "A {gender} speaker delivers a {style} explanation in a clear teaching voice.",
    "This recording features a {gender} voice with a {style} speaking style.",
    "A {style} narration presented by a {gender} teacher.",
    "A {gender} educator explains the topic using a {style} tone.",
    "A clear and {style} explanation spoken by a {gender} voice.",
    "This audio contains a {gender} speaker using a {style} delivery.",
    "A {style} teaching narration performed by a {gender} individual.",
    "A professional {gender} voice speaking in a {style} manner.",
    "A calm and informative {style} explanation from a {gender} speaker.",
    "A {gender} teacher presents the topic with a {style} approach.",
    "A {style} educational explanation delivered by a {gender} voice.",
    "A natural {gender} voice expressing a {style} teaching style.",
    "A {gender} narrator speaks with a {style} tone for learning purposes.",
    "A friendly {style} explanation provided by a {gender} speaker.",
    "A focused {style} teaching voice from a {gender} educator.",
    "This sample includes a {gender} voice using a {style} narration style.",
    "A structured {style} explanation spoken by a {gender} teacher.",
    "A {gender} speaker communicates the lesson in a {style} way.",
    "An educational {style} narration performed by a {gender} voice.",
    "A confident {gender} speaker delivering a {style} explanation.",
    "A simple and {style} teaching narration from a {gender} educator.",
    "A {gender} instructional voice with a {style} expression.",
    "A clear {style} lesson explained by a {gender} speaker.",
    "A composed {gender} voice presenting content in a {style} tone.",
    "A {style} learning-focused narration by a {gender} teacher.",
    "A professional educational explanation in a {style} voice by a {gender} speaker.",
    "A {gender} speaker delivers knowledge using a {style} teaching tone.",
    "A smooth and {style} explanation spoken by a {gender} voice.",
    "A {style} classroom-style narration from a {gender} educator.",
    "A direct and {style} explanation presented by a {gender} speaker."
]

# ======================= GENDER MODEL =======================
print("🔍 Loading gender classification model...")
gender_model_name = "prithivMLmods/Common-Voice-Geneder-Detection"
gender_model = Wav2Vec2ForSequenceClassification.from_pretrained(gender_model_name)
gender_processor = Wav2Vec2FeatureExtractor.from_pretrained(gender_model_name)
gender_id2label = {0: "female", 1: "male"}
gender_model.eval()

def predict_gender(audio_path):
    speech, _ = librosa.load(audio_path, sr=16000)
    inputs = gender_processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = gender_model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
    return gender_id2label[pred]

# ======================= WAV SAVER =======================
def save_wav(filename, raw_data, rate=24000):
    if isinstance(raw_data, str):
        raw_data = base64.b64decode(raw_data)
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(rate)
        wf.writeframes(raw_data)

# ======================= MAIN LOOP =======================
topics = [
    # --- SPACE & COSMOS ---
    "What makes planets round?",
    "How fast does Earth travel through space?",
    "Why does the Moon have phases?",
    "How do astronauts train on Earth?",
    "What is microgravity?",
    "Why do stars have different colors?",
    "How long does it take light to reach Earth?",
    "What happens if you cry in space?",
    "Why do astronauts wear helmets?",
    "What is a space vacuum?",
    "How do rockets escape Earth’s gravity?",
    "Why does the Moon always show the same side?",
    "What are meteorites made of?",
    "How do astronauts stay healthy in space?",
    "Why is the Sun so hot?",
    "What is cosmic radiation?",
    "How do planets form?",
    "Why does Mars have dust storms?",
    "What is a solar eclipse?",
    "How do space missions return to Earth?",
    "What is a dwarf planet?",
    "Why does Saturn have many moons?",
    "How do space telescopes work?",
    "What is interstellar space?",
    "Why is Uranus tilted?",
    "How do we name stars?",
    "What is a red giant star?",
    "How do astronauts exercise in space?",
    "Why does the Sun rise in the east?",
    "What is a space capsule?",
    "Why does space have no air?",
    "How do astronomers study planets?",
    "What is a gravity assist?",
    "Why does Mercury have extreme temperatures?",
    "How do we detect asteroids?",
    "What is a binary star?",
    "Why does the Moon affect tides?",
    "How do spacecraft land on other planets?",
    "What is a cosmic ray?",
    "Why is Jupiter’s gravity so strong?",

    # --- EARTH & GEOLOGY ---
    "How old is the Earth?",
    "Why does the Earth spin?",
    "How do continents move?",
    "What causes hot spots on Earth?",
    "Why do volcanoes form near oceans?",
    "How do mountains rise?",
    "Why does sand feel hot?",
    "How do rivers create valleys?",
    "What makes soil fertile?",
    "Why do rocks have layers?",
    "How do earthquakes release energy?",
    "Why do deserts expand?",
    "How does wind cause erosion?",
    "What makes clay sticky?",
    "Why do some rocks float?",
    "How do natural arches form?",
    "Why does land sink in some areas?",
    "How do caves get their shapes?",
    "Why do beaches have waves?",
    "How does ice shape landscapes?",
    "What causes landslides?",
    "Why do rivers flood?",
    "How does sediment build deltas?",
    "Why does lava glow?",
    "How do hot springs stay warm?",
    "Why do cliffs collapse?",
    "How does Earth recycle rocks?",
    "What causes sinkholes?",
    "Why do volcanoes sleep for years?",
    "How do rocks become smooth?",
    "Why do some mountains erupt?",
    "How do glaciers carve land?",
    "Why is Earth’s mantle hot?",
    "How do mineral crystals form?",
    "Why does land crack in droughts?",
    "How do fault lines form?",
    "Why do rocks sparkle?",
    "How does erosion affect farms?",
    "Why do islands sink over time?",
    "How does Earth cool itself?",

    # --- ANIMALS & WILDLIFE ---
    "How do animals adapt to cold?",
    "Why do birds build nests?",
    "How do fish breathe underwater?",
    "Why do animals migrate?",
    "How do insects survive winter?",
    "Why do mammals have fur?",
    "How do animals communicate silently?",
    "Why do some animals glow?",
    "How do predators hunt?",
    "Why do animals camouflage?",
    "How do baby animals learn?",
    "Why do animals groom themselves?",
    "How do animals find water?",
    "Why do reptiles bask in the sun?",
    "How do animals balance?",
    "Why do animals play fight?",
    "How do animals remember paths?",
    "Why do animals live in groups?",
    "How do animals sense storms?",
    "Why do animals have instincts?",
    "How do birds build strong nests?",
    "Why do animals mark territory?",
    "How do animals survive drought?",
    "Why do animals change color?",
    "How do animals heal wounds?",
    "Why do animals molt?",
    "How do nocturnal animals see?",
    "Why do animals stretch?",
    "How do animals smell danger?",
    "Why do animals nap often?",
    "How do animals use whiskers?",
    "Why do animals groom others?",
    "How do animals find mates?",
    "Why do animals defend babies?",
    "How do animals learn tricks?",
    "Why do animals have claws?",
    "How do animals share food?",
    "Why do animals migrate long distances?",
    "How do animals stay cool?",
    "Why do animals sleep differently?",

    # --- HUMAN BODY ---
    "Why does our heart never rest?",
    "How does blood carry oxygen?",
    "Why do muscles get sore?",
    "How does the brain learn?",
    "Why do we need sleep?",
    "How do bones heal themselves?",
    "Why do we breathe faster when scared?",
    "How does digestion start?",
    "Why does our body need water?",
    "How does skin protect us?",
    "Why do we feel hungry?",
    "How does the brain control movement?",
    "Why do we sweat?",
    "How does the body fight viruses?",
    "Why do eyes blink automatically?",
    "How does the body balance?",
    "Why do we feel emotions?",
    "How does memory form?",
    "Why do we feel pain?",
    "How does the body grow taller?",
    "Why does hair fall out?",
    "How does the body repair cells?",
    "Why do we feel fear?",
    "How does hearing work?",
    "Why do we need vitamins?",
    "How does taste work?",
    "Why do we need oxygen?",
    "How does the immune system learn?",
    "Why does the body need salt?",
    "How do nerves send signals?",
    "Why do we yawn together?",
    "How does vision form images?",
    "Why do we feel stress?",
    "How does posture work?",
    "Why does the heart beat rhythmically?",
    "How does the body remove waste?",
    "Why do we feel tired after eating?",
    "How does the brain control speech?",
    "Why do we get muscle cramps?",
    "How does the body stay warm?",

    # --- PHYSICS ---
    "Why does light travel so fast?",
    "How does motion start?",
    "Why does sound fade with distance?",
    "How does gravity pull objects?",
    "Why does friction create heat?",
    "How does energy transfer?",
    "Why does pressure increase underwater?",
    "How does momentum work?",
    "Why do objects fall straight down?",
    "How does acceleration change speed?",
    "Why does light bend?",
    "How does reflection work?",
    "Why does electricity flow?",
    "How does force change motion?",
    "Why does vibration create sound?",
    "How does heat spread?",
    "Why do waves carry energy?",
    "How does magnetism attract metal?",
    "Why does mass matter?",
    "How does inertia work?",
    "Why do objects resist motion?",
    "How does tension work?",
    "Why does sound travel faster in solids?",
    "How does pressure move liquids?",
    "Why do objects compress?",
    "How does energy transform?",
    "Why does speed depend on mass?",
    "How does equilibrium work?",
    "Why do objects oscillate?",
    "How does resonance work?",
    "Why does air resist motion?",
    "How does torque rotate objects?",
    "Why do objects spin?",
    "How does balance work?",
    "Why does temperature rise?",
    "How does light scatter?",
    "Why does sound vibrate air?",
    "How does force multiply?",
    "Why do objects bounce?",
    "How does pressure equalize?",

    # --- TECHNOLOGY & ENGINEERING ---
    "How do machines reduce effort?",
    "Why do gears change speed?",
    "How do engines convert fuel?",
    "Why do structures need supports?",
    "How do circuits control power?",
    "Why do machines vibrate?",
    "How do sensors detect motion?",
    "Why do motors spin?",
    "How do robots follow commands?",
    "Why do machines need cooling?",
    "How do levers lift weight?",
    "Why do pulleys reduce force?",
    "How do engines stay efficient?",
    "Why do devices need processors?",
    "How do signals transmit data?",
    "Why do machines need calibration?",
    "How do brakes control motion?",
    "Why do machines need lubrication?",
    "How do computers store data?",
    "Why do devices need updates?",
    "How do antennas receive signals?",
    "Why do machines wear out?",
    "How do sensors measure temperature?",
    "Why do engines need air?",
    "How do machines stay balanced?",
    "Why do systems need backups?",
    "How do chips process logic?",
    "Why do machines need power?",
    "How do controllers manage systems?",
    "Why do devices overheat?",
    "How do tools multiply force?",
    "Why do machines fail?",
    "How do mechanisms lock?",
    "Why do systems need testing?",
    "How do machines self-correct?",
    "Why do devices use software?",
    "How do signals encode data?",
    "Why do machines follow rules?",
    "How do systems synchronize?",
    "Why do machines automate tasks?",

    # --- FOOD & CHEMISTRY ---
    "Why does food change color when cooked?",
    "How does fermentation work?",
    "Why do spices smell strong?",
    "How does sugar preserve food?",
    "Why does fat carry flavor?",
    "How does heat change texture?",
    "Why does baking need timing?",
    "How do acids affect taste?",
    "Why does oil fry food?",
    "How does freezing slow bacteria?",
    "Why does salt draw water?",
    "How does caramelization work?",
    "Why does dough need kneading?",
    "How do enzymes change food?",
    "Why does food spoil?",
    "How does marinating soften meat?",
    "Why does food smell when hot?",
    "How does sugar crystallize?",
    "Why does milk curdle?",
    "How does cooking kill germs?",
    "Why do onions make you cry?",
    "How does yeast make bread rise?",
    "Why does chocolate melt in your mouth?",
    "How do bubbles form in soda?",
    "Why is honey so thick?",
    "How does vinegar clean things?",
    "Why does fruit ripen?",
    "How does caffeine keep you awake?",
    "Why does popcorn pop?",
    "How does soap remove grease?",

    # --- ENVIRONMENT & ECOLOGY ---
    "How do ecosystems stay balanced?",
    "Why do species depend on each other?",
    "How do plants clean air?",
    "Why does pollution harm wildlife?",
    "How do forests control climate?",
    "Why do wetlands prevent floods?",
    "How does recycling save energy?",
    "Why does biodiversity matter?",
    "How do oceans regulate temperature?",
    "Why do humans impact habitats?",
    "How does climate affect animals?",
    "Why do ecosystems collapse?",
    "How do food chains work?",
    "Why do invasive species spread?",
    "How does conservation help nature?",
    "Why do coral reefs die?",
    "How do trees store carbon?",
    "Why does soil erosion matter?",
    "How does pollution spread?",
    "Why does water scarcity happen?",
    "How does composting help the soil?",
    "Why do we need bees for food?",
    "How does plastic affect the ocean?",
    "Why do glaciers melt?",
    "How does the ozone layer protect us?",
    "Why do we use renewable energy?",
    "How does rain wash away pollutants?",
    "Why do urban areas feel hotter?",
    "How do seeds travel long distances?",
    "Why is clean water a human right?",

    # --- HISTORY & CIVILIZATION ---
    "Why did ancient people build pyramids?",
    "How did the Silk Road connect the world?",
    "Why did the Roman Empire collapse?",
    "How was paper invented?",
    "Why did explorers sail across oceans?",
    "How did the printing press change history?",
    "Why did the Industrial Revolution start?",
    "How were ancient cities designed?",
    "Why did civilizations settle near rivers?",
    "How did humans discover fire?",
    "Why did the Renaissance happen?",
    "How did codes and ciphers win wars?",
    "Why did knights wear armor?",
    "How did the Great Wall of China start?",
    "Why did the Maya civilization vanish?",
    "How did early humans migrate?",
    "Why did the Vikings travel so far?",
    "How was the first calendar made?",
    "Why did castles have moats?",
    "How did the steam engine change travel?",
    "Why did the Olympics begin?",
    "How did writing systems evolve?",
    "Why did nations sign the Magna Carta?",
    "How did the telephone change communication?",
    "Why was the Eiffel Tower built?",
    "How did agriculture lead to cities?",
    "Why did ancient Greeks value philosophy?",
    "How did the compass help sailors?",
    "Why did the Age of Enlightenment occur?",
    "How did railroads unify countries?",
    "Why did gold rushes happen?",
    "How did the Wright brothers fly?",
    "Why did the Apollo 11 mission matter?",
    "How did the internet begin?",
    "Why did the Berlin Wall fall?",
    "How did early doctors study the body?",
    "Why did trade routes create wealth?",
    "How did the Magna Carta influence laws?",
    "Why did the Victorian era change society?",
    "How did electricity change city life?",

    # --- MATHEMATICS & LOGIC ---
    "Why is zero a special number?",
    "How does the Fibonacci sequence appear in nature?",
    "Why do we use base-10 for counting?",
    "How do prime numbers protect data?",
    "Why is pi an infinite number?",
    "How does geometry help architects?",
    "Why do we use symbols for math?",
    "How do probability and luck differ?",
    "Why is the Pythagorean theorem useful?",
    "How do computers use binary?",
    "Why do circles have 360 degrees?",
    "How does algebra solve for unknowns?",
    "Why are fractals found in plants?",
    "How does statistics help predict trends?",
    "Why is the Golden Ratio beautiful?",
    "How do graphs visualize data?",
    "Why do we have negative numbers?",
    "How do decimals work?",
    "Why is symmetry important in math?",
    "How do logic puzzles train the brain?",
    "Why do we use metric and imperial units?",
    "How does calculus measure change?",
    "Why is the number 7 considered lucky?",
    "How do algorithms make decisions?",
    "Why are triangles the strongest shape?",
    "How do we calculate the area of a sphere?",
    "Why do prime numbers never end?",
    "How do ratios help in cooking?",
    "Why is math called the universal language?",
    "How do patterns help us learn?",

    # --- WEATHER & METEOROLOGY ---
    "What causes thunder and lightning?",
    "How do clouds form?",
    "Why is the sky blue?",
    "How do hurricanes get their names?",
    "Why does it smell like rain?",
    "How do rainbows form?",
    "Why is it colder at the poles?",
    "How do tornadoes spin?",
    "Why does wind blow?",
    "How does humidity affect temperature?",
    "Why does snow have different shapes?",
    "How do seasons change?",
    "Why is fog common in the morning?",
    "How do we predict the weather?",
    "Why are sunsets red and orange?",
    "How does air pressure affect us?",
    "Why do monsoons happen?",
    "How does the Jet Stream move?",
    "Why is hail different from snow?",
    "How do heat waves occur?",
    "Why does the wind howl?",
    "How do deserts stay dry?",
    "Why do clouds have different heights?",
    "How does lightning strike the ground?",
    "Why is it humid in the tropics?",
    "How does a barometer work?",
    "Why do we see our breath in winter?",
    "How do thunderstorms end?",
    "Why does the wind change direction?",
    "How does a drought start?",

    # --- PLANTS & BOTANY ---
    "How do plants eat sunlight?",
    "Why do leaves change color in fall?",
    "How do trees communicate through roots?",
    "Why do flowers have scents?",
    "How do cactus survive without water?",
    "Why do some plants eat insects?",
    "How do seeds know when to grow?",
    "Why do trees have bark?",
    "How does water travel up a tall tree?",
    "Why do weeds grow so fast?",
    "How do mushrooms differ from plants?",
    "Why do flowers bloom at different times?",
    "How do plants protect themselves?",
    "Why do roots grow downward?",
    "How do fruits develop?",
    "Why are some plants poisonous?",
    "How do mosses grow without soil?",
    "Why do leaves have veins?",
    "How do plants help the soil?",
    "Why do some trees stay green all year?",

    # --- PSYCHOLOGY & THE MIND ---
    "Why do we dream?",
    "How do habits form?",
    "Why do we feel deja vu?",
    "How does music affect mood?",
    "Why do we laugh?",
    "How do first impressions work?",
    "Why do we forget things?",
    "How does color influence our feelings?",
    "Why do we fear the dark?",
    "How does meditation calm the brain?",
    "Why do we procrastinate?",
    "How do we recognize faces?",
    "Why do some people love spicy food?",
    "How does exercise improve mental health?",
    "Why do we cry when we are happy?",
    "How do we learn a new language?",
    "Why do we feel butterflies in our stomach?",
    "How does focus work?",
    "Why do we enjoy scary movies?",
    "How does confidence help us succeed?",

    # --- CHEMISTRY & MATTER ---
    "What is an atom?",
    "Why does ice float on water?",
    "How do batteries store energy?",
    "Why does metal rust?",
    "How does fire burn?",
    "Why is gold so valuable?",
    "How do magnets work at a molecular level?",
    "Why is diamond so hard?",
    "How does soap kill germs?",
    "Why do neon signs glow?",
    "How does plastic melt?",
    "Why is water called the universal solvent?",
    "How do fireworks get their colors?",
    "Why does salt melt ice?",
    "How do acids and bases react?",
    "Why is oxygen vital for combustion?",
    "How does silver tarnish?",
    "Why do helium balloons float?",
    "How does rubber stretch?",
    "Why do some chemicals smell like rotten eggs?",

    # --- OCEANOGRAPHY & WATER ---
    "Why is the ocean salty?",
    "How deep is the deepest part of the ocean?",
    "Why is the sea blue?",
    "How do ocean currents move?",
    "Why do tides happen twice a day?",
    "How do whales sing?",
    "Why are coral reefs so colorful?",
    "How do underwater volcanoes erupt?",
    "Why is there more life near the surface?",
    "How do tsunamis start?",
    "Why do waves break near the shore?",
    "How do fish survive high pressure?",
    "Why are some parts of the ocean dead zones?",
    "How does evaporation work?",
    "Why is freshwater so rare?",
    "How do icebergs form?",
    "Why do sea shells have patterns?",
    "How do ships float if they are heavy?",
    "Why is the ocean floor so cold?",
    "How do rivers stay fresh?",

    # --- GENERAL CURIOSITY ---
    "How does a mirror reflect?",
    "Why is a week seven days?",
    "How do clocks keep time?",
    "Why do we use money?",
    "How does a lightbulb glow?",
    "Why do we have fingerprints?",
    "How does a parachute slow you down?",
    "Why is the Earth tilted?",
    "How do we hear echoes?",
    "Why is glass transparent?",
    "How does a vacuum cleaner work?",
    "Why do balls bounce?",
    "How does a compass find North?",
    "Why is lead heavy?",
    "How does a refrigerator stay cold?",
    "Why do we have different time zones?",
    "How does a microphone pick up sound?",
    "Why do we get hiccups?",
    "How does glue stick?",
    "Why is the sky dark at night?"
]

total_samples = 600
random.shuffle(topics)
final_topics = (topics * (total_samples // len(topics) + 1))[:total_samples]

print("🚀 Starting dataset generation...")

for i, topic in enumerate(final_topics, 1):
    style = random.choice(STYLES)
    voice = random.choice(VOICES)
    template = random.choice(TEMPLATES)

    sample_id = f"teacher_{i:04d}_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
    audio_path = os.path.join(VOICE_DIR, f"{sample_id}.wav")

    print(f"\n🎬 [{i}/{total_samples}] {topic} | Style: {style}")

    try:
        text_resp = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"As a teacher feeling {style}, explain '{topic}' in 2 simple sentences for a child."
        )
        teacher_text = text_resp.text.strip()

        temp_desc = template.format(gender="a", style=style)

        audio_resp = client.models.generate_content(
            model="gemini-2.5-flash-preview-tts",
            contents=f"{temp_desc}\n\n{teacher_text}",
            config=types.GenerateContentConfig(
                response_modalities=["AUDIO"],
                speech_config=types.SpeechConfig(
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(voice_name=voice)
                    )
                )
            )
        )

        audio_bytes = audio_resp.candidates[0].content.parts[0].inline_data.data
        save_wav(audio_path, audio_bytes)

        predicted_gender = predict_gender(audio_path)

        final_description = template.format(
            gender=predicted_gender,
            style=style
        )

        with open(METADATA_FILE, "a") as f:
            f.write(json.dumps({
                "audio_file": f"voices/{sample_id}.wav",
                "text": teacher_text,
                "description": final_description,
                "voice_name": voice,
                "style": style,
                "topic": topic,
                "gender": predicted_gender
            }) + "\n")

        print(f"✅ Saved | Gender Detected: {predicted_gender}")
        time.sleep(8)

    except Exception as e:
        if "quota" in str(e).lower() or "429" in str(e):
            rotate_key()
        else:
            print(f"❌ Error: {e}")

print("\n🎉 DONE — Clean dataset with gender + style only.")